In [1]:
import json
import numpy as np
from bokeh.plotting import figure
from bokeh.io import output_notebook, show
from bokeh.palettes import Category20_20
output_notebook()
colors = Category20_20

Loading BokehJS ...

In [2]:
info = json.loads(open('runs/1/info.json').read())
p = figure(title='CNN - MNIST', plot_width=600, plot_height=400)

c = 0
for k in info.keys():
    if k.startswith('uncertainty'):
        xy = np.array(info[k])
        area = np.trapz(y=xy[:, 1], x=xy[:, 0])
        p.line(xy[:, 0], xy[:, 1], legend='{0}: {1:.6f}'.format(k, area), color=colors[c], line_width=3)
        c += 1

p.xaxis.axis_label = 'data proportion'
p.yaxis.axis_label = 'accuracy'
p.legend.location = 'bottom_left'

show(p)

In [3]:
info = json.loads(open('runs/2/info.json').read())
p = figure(title='CNN - MNIST', plot_width=600, plot_height=400)

c = 0
for k in info.keys():
    if k.startswith('uncertainty'):
        xy = np.array(info[k])
        area = np.trapz(y=xy[:, 1], x=xy[:, 0])
        p.line(xy[:, 0], xy[:, 1], legend='{0}: {1:.6f}'.format(k, area), color=colors[c], line_width=3)
        c += 1

p.xaxis.axis_label = 'data proportion'
p.yaxis.axis_label = 'accuracy'
p.legend.location = 'bottom_left'

show(p)

In [4]:
info = json.loads(open('runs/3/info.json').read())
p = figure(title='MLP - Protein Structure', plot_width=800, plot_height=600)

c = 0
for k in info.keys():
    if k.startswith('uncertainty'):
        xy = np.array(info[k])
        area = np.trapz(y=xy[:, 1], x=xy[:, 0])
        p.line(xy[:, 0], xy[:, 1], legend='{0}: {1:.6f}'.format(k, area), color=colors[c], line_width=3)
        c += 1

p.xaxis.axis_label = 'data proportion'
p.yaxis.axis_label = 'RMSE'
p.legend.location = 'bottom_right'

show(p)